# Master pipeline notebook

This notebook is mainly meant to run the experimentation. 

The experimentation is divided into seperate parts according to machine learning process

The process involves: 
1. Data gathering and initial transformation
2. Data analysis
3. feature engineering 
4. Modelling 
5. Evaluation

In [1]:
import numpy as np
import pandas as pd
from data import gatherData
from pre_process_data import processData
from utils import load_config_file

In [2]:
config = load_config_file("config.yml")

## 1. Data gathering and transformation

This stage is mainly to gather the data. Combine them if necessary. Perform initial data wrangling, so that it is ready for 
data analysis

In [3]:
gather_data_obj = gatherData(config=config)

In [4]:
(
    lap_times_df,
    pit_stops_df,
    qualifying_df,
    races_df,
    results_df,
    sprint_results_df,
    status_df,
) = gather_data_obj.load_data()

In [6]:
pre_process_obj = processData(config=config)

master_lap_times_df = pre_process_obj.process_data(
    lap_times_df=lap_times_df,
    pit_stops_df=pit_stops_df,
    qualifying_df=qualifying_df,
    races_df=races_df,
    results_df=results_df,
    sprint_results_df=sprint_results_df,
    status_df=status_df,
)

master_lap_times_df

,raceId,driverId,lap,position,time,milliseconds,year,round,circuitId,name,date,number
0,841,20,1,1,1:38.109,98109,2011,1,1,Australian Grand Prix,2011-03-27,1
1,841,20,2,1,1:33.006,93006,2011,1,1,Australian Grand Prix,2011-03-27,1
2,841,20,3,1,1:32.713,92713,2011,1,1,Australian Grand Prix,2011-03-27,1
3,841,20,4,1,1:32.803,92803,2011,1,1,Australian Grand Prix,2011-03-27,1
4,841,20,5,1,1:32.342,92342,2011,1,1,Australian Grand Prix,2011-03-27,1
...,...,...,...,...,...,...,...,...,...,...,...,...
551737,1110,817,40,17,1:54.361,114361,2023,12,13,Belgian Grand Prix,2023-07-30,3
551738,1110,817,41,17,1:53.367,113367,2023,12,13,Belgian Grand Prix,2023-07-30,3
551739,1110,817,42,16,1:55.247,115247,2023,12,13,Belgian Grand Prix,2023-07-30,3
551740,1110,817,43,16,1:52.115,112115,2023,12,13,Belgian Grand Prix,2023-07-30,3
